# Bryan Examples

In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive

drive.mount(ROOT)           # we mount the drive at /content/drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
GIT_PATH = "https://github.com/ybchen97/filler_detection.git"
!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp

Cloning into 'filler_detection'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 4942 (delta 30), reused 30 (delta 15), pack-reused 4887
Receiving objects: 100% (4942/4942), 841.47 MiB | 30.80 MiB/s, done.
Resolving deltas: 100% (476/476), done.
mv: cannot stat './temp/*': No such file or directory


In [3]:
!pip install -r '/content/filler_detection/requirements.txt'

     |████████████████████████████████| 194kB 7.9MB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
     |████████████████████████████████| 163kB 46.3MB/s 
     |████████████████████████████████| 573kB 47.0MB/s 
     |████████████████████████████████| 9.5MB 26.1MB/s 
     |████████████████████████████████| 839kB 46.2MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 2.7MB 22.6MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 122kB 51.4MB/s 
     |████████████████████████████████| 788kB 47.6MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 43.2MB/s 
     |████████████████████████████████| 133kB 51.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 112kB 52.6MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
   

In [0]:
import random
import math
import sys
import os
import glob
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.silence import detect_silence, detect_nonsilent
from collections import defaultdict
from pydub.playback import play
from scipy.io import wavfile
from scipy import signal

In [0]:
# # For local
# AUDIO_IGNORED_EXAMPLES_DIRECTORY = "../ignored_audio_examples/"
# POSITIVE_DIRECTORY = AUDIO_IGNORED_EXAMPLES_DIRECTORY + "right/"
# BRYAN_LECTURES = AUDIO_IGNORED_EXAMPLES_DIRECTORY + "bryan_lectures/"
# TRAINING_SET_DIRECTORY = "../ignored_data/"

In [0]:
# For Colab
PROJ_PATH = '/content/drive/My Drive/filler_detection/'
POSITIVE_DIRECTORY = PROJ_PATH + "raw_data/positive_data/right/"
BRYAN_LECTURES = PROJ_PATH + "raw_data/bryan_lectures/"

In [0]:
POSITIVE_AUDIONAMES = [name for name in os.listdir(POSITIVE_DIRECTORY) if name.endswith("wav")]    

## Process Lecture
Cut out long periods of silence

In [0]:
def process_lecture(input_filepath, output_filepath, min_silence_len=3000, silence_thresh=-20, seek_step=3000):
    assert input_filepath.endswith('.mp4'), "Only processes mp4 videos!"
    
    # Import mp4
    segment = AudioSegment.from_file(input_filepath, "mp4")
    
    silent_ranges = detect_silence(segment, min_silence_len, silence_thresh, seek_step)
    print("Silent ranges in ms... {}".format(silent_ranges))
    
    # Trim silence
    # AudioSegments do not support `remove` functionality. Instead of removing silence, this code concatenates non-silence.
    # Internally its the same as `detect_nonsilence` uses `detect_silence` and takes the complement of the result.
    print("Concatenating nonsilent ranges...")
    nonsilent_ranges = detect_nonsilent(segment, min_silence_len, silence_thresh, seek_step) 
    result = AudioSegment.empty()
    for nonsilent_range in nonsilent_ranges:
        [start, end] = nonsilent_range
        result += segment[start: end]
        
    # Double check
    silent_ranges = detect_silence(result, min_silence_len, silence_thresh, seek_step)
    assert silent_ranges == [], "There are still silent ranges. Double check parameters of detect_nonsilent"
    
    # Set frame rate to 123000
    result = result.set_channels(1)
    result = result.set_frame_rate(123000)
    
    # Output as monochrome wav
    result.export(output_filepath, format='wav')
    print("Audio output in {}".format(output_filepath))

In [6]:
input_filepath = BRYAN_LECTURES + "train/concept_learning.mp4"
output_filepath = BRYAN_LECTURES + "train/concept_learning.wav"
process_lecture(input_filepath, output_filepath)

Silent ranges in ms... [[0, 96000], [99000, 108000], [120000, 123000], [153000, 156000], [345000, 348000], [462000, 465000], [528000, 531000], [537000, 540000], [564000, 567000], [582000, 585000], [612000, 615000], [738000, 741000], [813000, 816000], [837000, 843000], [975000, 978000], [1026000, 1029000], [1050000, 1071000], [1080000, 1083000], [1101000, 1110000], [1230000, 1233000], [1305000, 1308000], [1368000, 1371000], [1389000, 1392000], [1455000, 1461000], [1503000, 1506000], [1653000, 1656000], [1659000, 1662000], [1686000, 1689000], [1815000, 1818000], [1887000, 1890000], [1953000, 1962000], [2061000, 2064000], [2091000, 2094000], [2124000, 2127000], [2196000, 2202000], [2223000, 2235000], [2295000, 2298000], [2301000, 2304000], [2307000, 2310000], [2313000, 2316000], [2343000, 2346000], [2358000, 2364000], [2403000, 2406000], [2451000, 2454000], [2544000, 2550000], [2634000, 2637000], [2667000, 2670000], [2781000, 2784000], [2829000, 2832000], [2856000, 2859000], [2898000, 290

KeyboardInterrupt: ignored

## Create One Training Example

In [0]:
Tx = 5490 # Based on created training example
n_freq = 129 # Based on created training example
Ty = 1369 # Based on model.summary() in 1.4 with shape := (Tx, n_freq)
min_silence_len = 1000
silence_thres = -20
seek_step = 1000
mini_segment_duration = 15000 # Allowing mini_segment_duration > labelled_duration is useful when substituting silence with filler words which are too short, which causes segment to be padded with too much silence at the end.
labelled_duration = 10000 

In [0]:
# input_filepath = BRYAN_LECTURES + "bayes_inference_processed.wav"
# segment = AudioSegment.from_wav(input_filepath)

In [0]:
def make_silence_dict(segment):
    """ Returns dictionary mapping number of silence in the %labelled_duration (ms) of mini_segment to mini_segment (ms). 
    """
    silence_dict = defaultdict(set)
    for mini_segment_start in range(0, len(segment) - mini_segment_duration, mini_segment_duration):
        mini_segment = segment[mini_segment_start: mini_segment_start + mini_segment_duration]
        silent_ranges = detect_silence(mini_segment[:labelled_duration], min_silence_len, silence_thres, seek_step)
        silence_dict[len(silent_ranges)].add(mini_segment)
    return silence_dict

In [0]:
# silence_dict = make_silence_dict(segment)
# for k in silence_dict.keys():
#     print("{} silence has {} examples".format(k, len(silence_dict[k])))
    
# silence_count = 2
# mini_segment = silence_dict[silence_count].pop() 
# silence_dict[silence_count].add(mini_segment) 
# silent_ranges = detect_silence(mini_segment[:labelled_duration], min_silence_len, silence_thres, seek_step)
# print(silent_ranges)
# assert len(silent_ranges) == silence_count, "Silence count does not match number of silence in segment.  Check construction of silence dict."

In [0]:
# play(mini_segment)

In [0]:
def create_X(segment, output_filepath, save):
    file_handle = segment.export(output_filepath, format='wav')

    sample_rate, samples = wavfile.read(output_filepath)
    frequencies, times, x = signal.spectrogram(samples, sample_rate)
    
    if save: print("File was saved in {}".format(output_filepath))
    else: os.remove(output_filepath)
        
    return frequencies, times, x

In [0]:
def insert_ones(y, segment_end_ms):
    """
    Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
    should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
    50 followinf labels should be ones.
    
    
    Arguments:
    y -- numpy array of shape (1, Ty), the labels of the training example
    segment_end_ms -- the end time of the segment in ms
    
    Returns:
    y -- updated labels
    """
    # duration of the background (in terms of spectrogram time-steps)
    segment_end_y = int(segment_end_ms * Ty / 10000.0)
    # Add 1 to the correct index in the background label (y)
    ### START CODE HERE ### (≈ 3 lines)
    for i in range(segment_end_y + 1, segment_end_y + 51):
        if i < Ty:
            y[0, i] = 1
    ### END CODE HERE ###
    
    return y

In [0]:
def create_Y(inserted_points):
    # Initialize y (label vector) of zeros (≈ 1 line)
    y = np.zeros((1, Ty))
    
    for point in inserted_points:
        y = insert_ones(y, point)
        
    assert y.shape == (1, Ty), "y shape needs to follow Ty!"
    
    return y

In [0]:
def random_positive(positive_audionames, debug = False):
    """
    Given a list of random positive audio names
    Return a randomly selected positive segment
    """
    # generate a random audio
    random_positive = np.random.choice(positive_audionames, 1, replace=False)[0]
    if debug:
        print("Selecting positive file randomly:\n    - {}".format(random_positive))
    # load audio
    global POSITIVE_DIRECTORY
    link = POSITIVE_DIRECTORY + random_positive
    return AudioSegment.from_wav(link)

In [0]:
def sub_filler(segment, debug = False):
    """ Inserts filler in every >= 1s pause in segment
    """
    
    result = AudioSegment.empty()
    inserted_points = []
    
    # if there's silence at the start, add filler word there
    silent_ranges = detect_silence(segment, min_silence_len, silence_thres, seek_step)
    first_silence_start = silent_ranges[0][0]
    if first_silence_start == 0:
        if debug: print("Filler added at {} ms".format(result.duration_seconds * 1000))
        result = random_positive(POSITIVE_AUDIONAMES, debug)
        inserted_points.append(result.duration_seconds * 1000)
    
    # adds filler word between nonsilent range
    nonsilent_ranges = detect_nonsilent(segment, min_silence_len, silence_thres, seek_step)
    for nonsilent_range in nonsilent_ranges: # adds filler after nonsilent ranges
        [start, end] = nonsilent_range
        result += segment[start:end]
        
        if nonsilent_range == nonsilent_ranges[-1]: continue # skip last
        if debug: print("Filler added at {} ms".format(result.duration_seconds * 1000))
        result += random_positive(POSITIVE_AUDIONAMES, debug)
        inserted_points.append(result.duration_seconds * 1000)
    
    # if there's silence at the end, add filler word there
    last_silence_end = silent_ranges[-1][-1]
    if last_silence_end == segment.duration_seconds * 1000:
        if debug: print("Filler added at {} ms".format(result.duration_seconds * 1000))
        result += random_positive(POSITIVE_AUDIONAMES, debug)
        inserted_points.append(result.duration_seconds * 1000)
    
    return result, inserted_points

In [0]:
# # Larry's filler word
# larry_segment = AudioSegment.from_wav("../raw_data/human_cont/1.wav")
# output_filepath = AUDIO_IGNORED_EXAMPLES_DIRECTORY + "temp.wav"
# inserted_segment, inserted_points = sub_filler(larry_segment, True)
# inserted_segment.export(output_filepath, format='wav')
# ipd.Audio(output_filepath)

In [0]:
def process_audio(segment, duration):
    # Trim or pad audio segment to %duration
    segment = segment[:duration]
    padding = AudioSegment.silent(duration)
    segment = padding.overlay(segment)
    # Set frame rate to 123000
    segment = segment.set_channels(1)
    segment = segment.set_frame_rate(123000)
    # Fade in: address model predicting 1 at start
    segment = segment.fade_in(1000)
    
    assert math.ceil(segment.duration_seconds) == duration / 1000, "segment needs to be exactly 10s long."
    return segment

In [0]:
def create_training_example(silence_dict, num_positives, output_filepath, save=False, debug=False):
    silence_count = num_positives
    inserted_points= []
    segment = silence_dict[silence_count].pop()
    if silence_count > 0: segment, inserted_points = sub_filler(segment, debug)
    segment = process_audio(segment, 10000)
    frequencies, times, x = create_X(segment, output_filepath, save)
    y = create_Y(inserted_points)
    
    return frequencies, times, x, y, inserted_points 

In [0]:
# output_filepath = "temp.wav"
# frequencies, times, x, y, inserted_points = create_training_example(silence_dict, 2, output_filepath, save=True, debug=True)
# print("x: {}".format(x.shape))
# print("y: {}".format(y.shape))
# plt.plot(y[0])
# print(inserted_points)
# ipd.Audio(output_filepath)

## Dataset Generation

In [0]:
def grep_wav(input_dir):
    """ Gets all .wav in input directory and returns them as a list.
    """
    wavs = []
    for file in os.listdir(input_dir):
        if file.endswith(".wav"): # lecture
            print("Adding wav: {}".format(input_dir + file))
            wavs.append(input_dir + file)
    return wavs

In [0]:
# lecs = grep_wav(BRYAN_LECTURES + "train/")

In [0]:
# input_prefix refers to the input_prefix naming of output audio files
def create_X_Y(size, start, mean_positives, variance, input_prefix, lecs, save=False, debug=False):
    i, X, Y, c = start, [], [], defaultdict(int)

    lec = lecs.pop()
    print("Processing lecture: {}...".format(lec))
    segment = AudioSegment.from_wav(lec)
    silence_dict = make_silence_dict(segment)
    for k in silence_dict.keys(): # check number of examples available
        print("{} silence has {} examples".format(k, len(silence_dict[k])))

    i = start
    while i < start + size:
        output_filepath = input_prefix + "_" + str(i) + ".wav"
        num_positives = int(random.gauss(mean_positives, variance))
        while num_positives > 3 or num_positives < 0: # hacky fix to prevent extreme positives
            num_positives = int(random.gauss(mean_positives, 1))
        try:
            print(output_filepath)
            _, _, x, y, _ = create_training_example(silence_dict, num_positives, output_filepath, save, debug)
            c[num_positives] += 1
        except KeyError:
            print("Ran out of examples for {} positives.".format(num_positives))
            if lecs: 
                lec = lecs.pop()
                print("Processing lecture: {}...".format(lec))
                segment = AudioSegment.from_wav(lec)
                silence_dict = make_silence_dict(segment)
                for k in silence_dict.keys(): # check number of examples available
                    print("{} silence has {} examples".format(k, len(silence_dict[k])))
                continue
            else:
                print ("Ran out of lectures.")
                break
        x = np.transpose(x)
        y = np.transpose(y)
        X.append(x)
        Y.append(y)
        print("Created example {}...".format(str(i)))
        i += 1
    for k in c.keys(): # check number of examples available
        print("{} positives has {} examples".format(k, c[k]))
    return (np.array(X), np.array(Y))

In [35]:
size = 100
start = 1
mean_positives = 2
variance = 1
input_prefix = BRYAN_LECTURES + "test/audio_examples/test"
lecs = grep_wav(BRYAN_LECTURES + "test/")
X, Y = create_X_Y(size, start, mean_positives, variance, input_prefix, lecs, save = True)
print("X:{} Y:{}".format(X.shape, Y.shape))

Adding wav: /content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/hypothesis_space.wav
Processing lecture: /content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/hypothesis_space.wav...
2 silence has 103 examples
1 silence has 68 examples
3 silence has 36 examples
0 silence has 22 examples
4 silence has 6 examples
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/audio_examples/test_1.wav
Created example 1...
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/audio_examples/test_2.wav
Created example 2...
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/audio_examples/test_3.wav
Created example 3...
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/audio_examples/test_4.wav
Created example 4...
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/test/audio_examples/test_5.wav
Created example 5...
/content/drive/My Drive/filler_detection/raw_data/bryan_lectures/tes

In [0]:
SAVE_DIRECTORY = PROJ_PATH + "data/test/bryan_synthesised/"
np.save(SAVE_DIRECTORY + "X_test.npy", X)
np.save(SAVE_DIRECTORY + "Y_test.npy", Y)